# notebook exploring how to make light

In [174]:
import numpy as np
from numba import jit
import time

In [175]:
# point of light
origin = [0, 0]

# custom anchor coords
anchor_coords = np.array([[10, 12], [11, 12]])
anchor_coords2 = np.array([
    [0, 0], 
    [1, 2], 
    [2, 4],
    [3, 4], 
    [10, 13], 
    [10, 12], 
    [11, 12]])

# random anchor coods
rand_N = 10000
random_coords = np.random.randint(0, 100, (rand_N, 2))

# Functions

In [176]:
# old

def anchor_to_square(anchors):
    """get vertex positions for a square from anchor"""
    template = np.array([[[0, 0], [0, 1], [1, 1], [1, 0]]])
    square = np.repeat(template, repeats=len(anchors), axis=0)
    for i, anchor in enumerate(anchors):
        square[i] += anchor
    return square


def squares_to_edgelists(squares):
    """
    using [[0, 0], [0, 1], [1, 1], [1, 0]]
    doing it this way allows for deleting common edges
    """
    edge_by_square = np.zeros((*squares.shape, 2))
    for i, square in enumerate(squares):
        edge_by_square[i] = np.array([
            [square[0], square[1]],
            [square[1], square[2]],
            [square[3], square[2]],
            [square[0], square[3]]])
    return edge_by_square


def squares_to_edgelists2(squares):
    """
    using [[0, 0], [0, 1], [1, 1], [1, 0]]
    doing it this way allows for deleting common edges
    """
    edge_by_square = np.zeros((len(squares), 4, 4))
    for i, square in enumerate(squares):
        edge_by_square[i] = np.array([
            [*square[0], *square[1]],
            [*square[1], *square[2]],
            [*square[3], *square[2]],
            [*square[0], *square[3]]])
    return edge_by_square


### First turn anchor points to lists of edges

index edges as follows, in order to be able to remove repeating edges later

    ↑ ---> ↑
    |      |
    | ---> |

In [177]:
def anchors_to_edgelists(anchors):
    """ edges must be done like this to be able to remove duplicates
    [0, 0, 0, 1] ^
    [0, 1, 1, 1] ->
    [1, 0, 1, 1] ^
    [0, 0, 1, 0] ->
    """
    edge_by_anchor = np.zeros((len(anchors), 4, 4))
    for i, a in enumerate(anchors):
        edge_by_anchor[i] = np.array([
            [a[0],   a[1],   a[0],   a[1]+1],
            [a[0],   a[1]+1, a[0]+1, a[1]+1],
            [a[0]+1, a[1],   a[0]+1, a[1]+1],
            [a[0],   a[1],   a[0]+1, a[1]]])
    return edge_by_anchor

In [178]:
# run
edges_by_anchor = anchors_to_edgelists(anchor_coords2)
print("using format: [x1,y1, x2,y2], e.g.")
print(edges_by_anchor[2:4])

using format: [x1,y1, x2,y2], e.g.
[[[2. 4. 2. 5.]
  [2. 5. 3. 5.]
  [3. 4. 3. 5.]
  [2. 4. 3. 4.]]

 [[3. 4. 3. 5.]
  [3. 5. 4. 5.]
  [4. 4. 4. 5.]
  [3. 4. 4. 4.]]]


### Then determine which edges are hidden from the light

#### Calculate distances for all the edges

In [179]:
def distance(point1, point2):
    return np.hypot((point2[1]-point1[1]), (point2[0]-point1[0]))

def edge_distances(origin, edges):
    edge_dists = np.zeros((len(edges), 2))
    for i, edge in enumerate(edges):
        edge_dists[i] = [distance(origin, edge[:2]),
                         distance(origin, edge[2:])]
    return edge_dists

In [180]:
# run
edge_dists = edge_distances(origin, edges_by_anchor[0])
print(edge_dists)

[[0.         1.        ]
 [1.         1.41421356]
 [1.         1.41421356]
 [0.         1.        ]]


#### Remove edges where their max is less than another one's min

In [181]:
def remove_shadow_edges(origin, edges):
    edge_dists = edge_distances(origin, edges)
    remove_idx = [True for x in edge_dists]
    maxs = edge_dists.max(1)
    mins = edge_dists.min(1)
    for vmin in mins:
        for i, vmax in enumerate(maxs):
            if vmin >= vmax:
                remove_idx[i] = False
    # # debug
    # print(remove_idx)
    # print(edge_dists[remove_idx])
    return edges[remove_idx]

In [182]:
# run
light_edges = remove_shadow_edges(origin, edges_by_anchor[0])
print(light_edges)

[[0. 1. 1. 1.]
 [1. 0. 1. 1.]]


### Accounting for repeats

In [183]:
# print(edges_by_anchor[:2])
all_light_edges = []
for sq in edges_by_anchor:
    light_edges = remove_shadow_edges(origin, sq)
    for edge in light_edges:
        all_light_edges.append(edge)
all_light_edges = np.array(all_light_edges)
print(all_light_edges)
print(all_light_edges.shape)
    

[[ 0.  1.  1.  1.]
 [ 1.  0.  1.  1.]
 [ 1.  3.  2.  3.]
 [ 2.  2.  2.  3.]
 [ 2.  5.  3.  5.]
 [ 3.  4.  3.  5.]
 [ 3.  5.  4.  5.]
 [ 4.  4.  4.  5.]
 [10. 14. 11. 14.]
 [11. 13. 11. 14.]
 [10. 13. 11. 13.]
 [11. 12. 11. 13.]
 [11. 13. 12. 13.]
 [12. 12. 12. 13.]]
(14, 4)


In [184]:
def rm_inside_edges(all_edges):
    uniq, counts = np.unique(all_edges, return_counts=True, axis=0)
    repeat_idx = np.nonzero(counts-1)[0]
    external_only = np.delete(uniq, repeat_idx, axis=0)
    return external_only

all_external_light_edges = rm_inside_edges(all_light_edges)
print(all_external_light_edges)
print(all_external_light_edges.shape)

# change edges into verteces
print(all_external_light_edges.reshape((len(all_external_light_edges)*2, 2)))

[[ 0.  1.  1.  1.]
 [ 1.  0.  1.  1.]
 [ 1.  3.  2.  3.]
 [ 2.  2.  2.  3.]
 [ 2.  5.  3.  5.]
 [ 3.  4.  3.  5.]
 [ 3.  5.  4.  5.]
 [ 4.  4.  4.  5.]
 [10. 13. 11. 13.]
 [10. 14. 11. 14.]
 [11. 12. 11. 13.]
 [11. 13. 11. 14.]
 [11. 13. 12. 13.]
 [12. 12. 12. 13.]]
(14, 4)
[[ 0.  1.]
 [ 1.  1.]
 [ 1.  0.]
 [ 1.  1.]
 [ 1.  3.]
 [ 2.  3.]
 [ 2.  2.]
 [ 2.  3.]
 [ 2.  5.]
 [ 3.  5.]
 [ 3.  4.]
 [ 3.  5.]
 [ 3.  5.]
 [ 4.  5.]
 [ 4.  4.]
 [ 4.  5.]
 [10. 13.]
 [11. 13.]
 [10. 14.]
 [11. 14.]
 [11. 12.]
 [11. 13.]
 [11. 13.]
 [11. 14.]
 [11. 13.]
 [12. 13.]
 [12. 12.]
 [12. 13.]]


## put everything together

In [185]:
def get_light_verteces(origin, obj_anchors):
    edges_by_anchor = anchors_to_edgelists(obj_anchors)
    all_light_edges = []
    for sq in edges_by_anchor:
        light_edges = remove_shadow_edges(origin, sq)
        ext_light_edges = rm_inside_edges(light_edges)
        for edge in ext_light_edges:
            all_light_edges.append(edge)
    all_light_edges = np.array(all_light_edges)
    print(all_light_edges)
    return all_light_edges.reshape((len(all_light_edges)*2, 2))

In [186]:
reshaped = get_light_verteces(origin, anchor_coords2)
print(reshaped)


[[ 0.  1.  1.  1.]
 [ 1.  0.  1.  1.]
 [ 1.  3.  2.  3.]
 [ 2.  2.  2.  3.]
 [ 2.  5.  3.  5.]
 [ 3.  4.  3.  5.]
 [ 3.  5.  4.  5.]
 [ 4.  4.  4.  5.]
 [10. 14. 11. 14.]
 [11. 13. 11. 14.]
 [10. 13. 11. 13.]
 [11. 12. 11. 13.]
 [11. 13. 12. 13.]
 [12. 12. 12. 13.]]
[[ 0.  1.]
 [ 1.  1.]
 [ 1.  0.]
 [ 1.  1.]
 [ 1.  3.]
 [ 2.  3.]
 [ 2.  2.]
 [ 2.  3.]
 [ 2.  5.]
 [ 3.  5.]
 [ 3.  4.]
 [ 3.  5.]
 [ 3.  5.]
 [ 4.  5.]
 [ 4.  4.]
 [ 4.  5.]
 [10. 14.]
 [11. 14.]
 [11. 13.]
 [11. 14.]
 [10. 13.]
 [11. 13.]
 [11. 12.]
 [11. 13.]
 [11. 13.]
 [12. 13.]
 [12. 12.]
 [12. 13.]]


# time tests

In [187]:
# run
t0 = time.time()
t1 = time.time()
print(t1-t0)

0.0


# Python Lessons

In [188]:
# boolean arrays as numpy indices
a = np.array([False, True, False, True])
b = np.array([1,2,3,4])
print(b[a])

[2 4]


In [189]:
# Repeats with Numpy
a = np.array([[2, 3], [5, 6], [7, 9]])
r_a = np.repeat(a, repeats=2, axis=0)
r_a = np.repeat(r_a, repeats=2, axis=1)
print(r_a)

[[2 2 3 3]
 [2 2 3 3]
 [5 5 6 6]
 [5 5 6 6]
 [7 7 9 9]
 [7 7 9 9]]


In [191]:
# numpy min of array
a = np.array([[0,1], [5,2], [100, 64]])
print(a.min(1))
print(a.max(1))

[ 0  2 64]
[  1   5 100]


# Experimentation

In [ ]:
# killing repeats
eee = np.array([
    [[0, 0], [0, 0]],
    [[0, 0], [0, 0]],
    [[1, 2], [2, 4]],
    [[1, 2], [2, 4]],
    [[2, 2], [2, 2]]])
b, cnts = np.unique(eee, return_counts=True, axis=0)
remove_idx = np.nonzero(cnts-1)[0]
cleaned = np.delete(b, remove_idx, axis=0)
print(b, "\n--------")
print(remove_idx, "\n--------")
print(cleaned)


[[[0 0]
  [0 0]]

 [[1 2]
  [2 4]]

 [[2 2]
  [2 2]]] 
--------
[0 1] 
--------
[[[2 2]
  [2 2]]]


In [206]:
orgn = np.array([10, 10])

edge_array = np.array([
    [11, 12, 11, 11],
    [11, 12, 11, 11],
    [11, 12, 11, 11],
    [11, 12, 11, 11],
])

def circle_practice(edge_array):
    for a in edge_array:
        print("----")
        a1 = a[:2]
        a2 = a[2:]

        step1 = a1 - orgn
        step2 = a2 - orgn
        print(step1, step2)

        slope1 = step1[1]/step1[0]
        slope2 = step2[1]/step2[0]
        print(slope1, slope2)

        rad1 = np.degrees(np.sin(slope1))
        rad2 = np.degrees(np.sin(slope2))
        print(rad1)
        print(rad2)

[1 2] [1 1]
2.0 1.0
52.098904879217365
48.21273601220948


In [232]:
orgn = np.array([10, 10])
point_array = np.array([
    [10, 11],
    [11, 11],
    [11, 10],
    [11,  9],
    [10,  9],
    [9,   9],
    [9,  10],
    [9,  11]])

pa = point_array.T

rads = np.arctan2(pa[0]-orgn[0], pa[1]-orgn[1])
degs = np.degrees(rads)
full_degs = degs + (degs<0)*360
print(rads)
print(degs)
print(full_degs)

# def circle_practice(origin, point_array):
#     for a in point_array:
#         step = a - origin
#         # print(step[1])
#         rads = np.arctan(step[0]/step[1])
#         degs = np.degrees(rads)
#         if step[1] < 0:
#             degs += 180
#         if step[1] == 0:
#             print(f"div by 0")
#         print(step, degs, rads, sep=' | ')

# circle_practice(orgn, point_array)

[ 0.          0.78539816  1.57079633  2.35619449  3.14159265 -2.35619449
 -1.57079633 -0.78539816]
[   0.   45.   90.  135.  180. -135.  -90.  -45.]
[  0.  45.  90. 135. 180. 225. 270. 315.]


In [248]:
segments = []
closest_edges = np.array([
    [0,   90 ],
    [45,  135],
    [30,  60 ],
    [0,   10 ],
    [355, 360]])

for i, entry in closest_edges:
    lower = True
    upper = True
    for seg in segments:
        if seg[0] <= entry[0] <= seg[1]:
            lower = False
        if seg[0] <= entry[1] <= seg[1]:
            upper = False
    if lower or upper:
        segments.append(entry)


print(np.array(segments))

IndexError: invalid index to scalar variable.

In [263]:
closest_edges = np.array([
    [[0,   90],
    [45,  135]],
    [[30,  60],
    [0,   10]],
    [[20, 30],
    [355, 360]]])
a = np.concatenate(closest_edges)
print(a)


[[  0  90]
 [ 45 135]
 [ 30  60]
 [  0  10]
 [ 20  30]
 [355 360]]


In [22]:
import numpy as np
line = np.array([[1,1,1,1,1,1], [1,1,1,1,1,1]])
rads = np.array([[1, 3, 5], [0, 2, 4]])
print(np.cross(rads, line))

ValueError: incompatible dimensions for cross product
(dimension must be 2 or 3)